# Testing Playground for r5py

In [3]:
import r5py

## Sample Datasets Provided by r5py themselves

### Helsinki

In [4]:
import r5py.sampledata.helsinki
import geopandas
import shapely

In [5]:
population_grid = geopandas.read_file(r5py.sampledata.helsinki.population_grid)

railway_station = geopandas.GeoDataFrame(
    {
        "id": ["railway_station"],
        "geometry": [shapely.Point(24.94152, 60.17066)]
    },
    crs="EPSG:4326",
)

In [6]:
overview_map = population_grid.explore("population", cmap="Reds")
overview_map = railway_station.explore(m=overview_map, marker_type="marker")
overview_map

In [7]:
transport_network = r5py.TransportNetwork(
    r5py.sampledata.helsinki.osm_pbf,
    [r5py.sampledata.helsinki.gtfs],
)

In [8]:
import datetime

origins = population_grid.copy()
origins.geometry = origins.geometry.centroid

destinations = railway_station.copy()

travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    departure=datetime.datetime(2022, 2, 22, 8, 30),
    transport_modes=[
        r5py.TransportMode.TRANSIT,
        r5py.TransportMode.WALK,
    ],
)

/tmp/ipykernel_2788/3435707801.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  origins.geometry = origins.geometry.centroid


In [9]:
travel_times = travel_time_matrix_computer.compute_travel_times()
travel_times.head()

,from_id,to_id,travel_time
0,0,railway_station,16
1,1,railway_station,18
2,2,railway_station,20
3,3,railway_station,22
4,4,railway_station,18


In [10]:
travel_times = population_grid.merge(travel_times, left_on="id", right_on="from_id")
travel_times.head()

,id,population,geometry,from_id,to_id,travel_time
0,0,389,"POLYGON ((24.90545 60.16086, 24.90545 60.16311...",0,railway_station,16
1,1,296,"POLYGON ((24.90546 60.15862, 24.90545 60.16086...",1,railway_station,18
2,2,636,"POLYGON ((24.90547 60.15638, 24.90546 60.15862...",2,railway_station,20
3,3,1476,"POLYGON ((24.90547 60.15413, 24.90547 60.15638...",3,railway_station,22
4,4,23,"POLYGON ((24.90994 60.16535, 24.90994 60.16760...",4,railway_station,18


In [11]:
travel_times.explore("travel_time", cmap="Greens")

## Acquiring and Preprocessing Datasets

### Road Networks from OSM

In [12]:
import zipfile
import pandas as pd


In [13]:
with zipfile.ZipFile('/home/emily/thesis_BA/data/gtfs/2023_rnv_gtfs.zip') as gtfs:
        with gtfs.open('stops.txt') as stops_file:
            stops_df = pd.read_table(stops_file, sep=",")
stops_df

,stop_id,stop_name,stop_lat,stop_lon,location_type,platform_code
0,11601,Bad Dürkheim Bahnhof,49.461511,8.170402,0,1
1,13401,Bad Dürkheim Ost,49.463437,8.185054,0,1
2,13402,Bad Dürkheim Ost,49.463315,8.184245,0,2
3,51501,Dossenheim Bahnhof,49.448191,8.670171,0,1
4,51502,Dossenheim Bahnhof,49.448553,8.669533,0,2
...,...,...,...,...,...,...
2074,752112,Rheinallee Süd,49.473887,8.455357,0,B
2075,752212,Drehbrücke,49.470350,8.452832,0,A
2076,752611,Rheinallee Nord,49.476560,8.455411,0,A
2077,814711,MA Waldorfschule,49.449312,8.473036,0,A


In [14]:
stops_gdf = geopandas.GeoDataFrame(
    stops_df, geometry=geopandas.points_from_xy(stops_df.stop_lon, stops_df.stop_lat), crs="EPSG:4326"
)
stops_gdf


,stop_id,stop_name,stop_lat,stop_lon,location_type,platform_code,geometry
0,11601,Bad Dürkheim Bahnhof,49.461511,8.170402,0,1,POINT (8.17040 49.46151)
1,13401,Bad Dürkheim Ost,49.463437,8.185054,0,1,POINT (8.18505 49.46344)
2,13402,Bad Dürkheim Ost,49.463315,8.184245,0,2,POINT (8.18425 49.46332)
3,51501,Dossenheim Bahnhof,49.448191,8.670171,0,1,POINT (8.67017 49.44819)
4,51502,Dossenheim Bahnhof,49.448553,8.669533,0,2,POINT (8.66953 49.44855)
...,...,...,...,...,...,...,...
2074,752112,Rheinallee Süd,49.473887,8.455357,0,B,POINT (8.45536 49.47389)
2075,752212,Drehbrücke,49.470350,8.452832,0,A,POINT (8.45283 49.47035)
2076,752611,Rheinallee Nord,49.476560,8.455411,0,A,POINT (8.45541 49.47656)
2077,814711,MA Waldorfschule,49.449312,8.473036,0,A,POINT (8.47304 49.44931)


In [15]:
stops_gdf.explore()

In [16]:
geofabrik_available = geopandas.read_file('/home/emily/thesis_BA/data/geofabrik_downloadindex.json')

#finding smallest available set covering gtfs feed
matching_datasets = geofabrik_available.contains(stops_gdf.unary_union)
coverage = geofabrik_available[matching_datasets]['geometry'].area
smallest = coverage.idxmin()
preferred_set = geofabrik_available.iloc[smallest]['id']

/tmp/ipykernel_2788/1406673922.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coverage = geofabrik_available[matching_datasets]['geometry'].area


In [17]:
from pathlib import Path
#check if parent dataset already downloaded
parent_set = geofabrik_available.iloc[smallest]['parent']
osm_data = None
for path in Path('/home/emily/thesis_BA/data/osm_data').glob(f"{parent_set}*"):
    osm_data = path

In [18]:
import pyrosm
if osm_data is None:
    osm_data = pyrosm.get_data(dataset=preferred_set, directory='/home/emily/thesis_BA/data/osm_data')

In [19]:
stop_bounds = stops_gdf.total_bounds
left, bottom, right, top = stop_bounds
type(top)

numpy.float64

In [20]:
'''
import subprocess

#cropping pbf to bounding box with osmosis
subprocess.run(['osmosis', '--read-pbf', f'file={osm_data}', '--bounding-box', f'top={top}', f'left={left}', f'bottom={bottom}', f'right={right}', 'completeWays=yes', '--write-pbf', f'file=/home/emily/thesis_BA/data/osm_data/test.osm.pbf'])
'''

"\nimport subprocess\n\n#cropping pbf to bounding box with osmosis\nsubprocess.run(['osmosis', '--read-pbf', f'file={osm_data}', '--bounding-box', f'top={top}', f'left={left}', f'bottom={bottom}', f'right={right}', 'completeWays=yes', '--write-pbf', f'file=/home/emily/thesis_BA/data/osm_data/test.osm.pbf'])\n"

this takes sooooooooo long

In [2]:
transport_network_rnv = r5py.TransportNetwork(
    osm_pbf='/home/emily/thesis_BA/data/osm_data/test.osm.pbf',
    gtfs='/home/emily/thesis_BA/data/gtfs/2023_rnv_gtfs.zip',
)

NameError: name 'r5py' is not defined

need to extract some stuff from the pbf too to have destinations

In [22]:
osm_data_obj = pyrosm.pyrosm.OSM('/home/emily/thesis_BA/data/osm_data/test.osm.pbf')
osm_data_obj.get_boundaries()

,visible,boundary,id,timestamp,version,tags,osm_type,geometry,name,ref,website,admin_level,border_type,changeset
0,False,administrative,162319720,1555672285,3,"{""note"":""Gemarkungsgrenze Flomersheim""}",way,"MULTILINESTRING ((8.33122 49.52966, 8.33389 49...",NaN,NaN,NaN,NaN,NaN,NaN
1,False,administrative,162319721,1555672285,4,"{""note"":""Gemarkungsgrenze M\u00F6rsch""}",way,"MULTILINESTRING ((8.36367 49.54444, 8.36472 49...",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,administrative,639343,1695675901,300,"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,"MULTIPOLYGON (((8.60334 49.59385, 8.75477 49.4...",Regierungsbezirk Karlsruhe,None,None,5,None,0.0
3,NaN,administrative,8462063,1701029748,472,"{""ISO3166-2"":""DE-RP"",""TMC:cid_58:tabcd_1:Class...",relation,"MULTIPOLYGON (((8.42250 49.58372, 8.42303 49.5...",Rheinland-Pfalz,RLP,None,4,state,0.0
4,NaN,administrative,8524410,1695967346,38,"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,"POLYGON ((8.29988 49.47387, 8.29983 49.47397, ...",Ludwigshafen am Rhein,None,None,6,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,NaN,administrative,34943106648,1693845044,2,"{""description"":""Das Gebiet der Gemarkung L\u00...",relation,"POLYGON ((8.72528 49.54759, 8.72590 49.54779, ...",Löhrbach,None,None,9,None,0.0
241,NaN,administrative,34959371635,1693551353,1,"{""description"":""Das Gebiet des Ortsteils Reise...",relation,"POLYGON ((8.71938 49.58355, 8.71936 49.58339, ...",Reisen,None,None,9,None,0.0
242,NaN,administrative,34975636637,1693551802,1,"{""description"":""Das Gebiet des Ortsteils Niede...",relation,"POLYGON ((8.71074 49.58194, 8.71043 49.58190, ...",Nieder-Liebersbach,None,None,9,None,0.0
243,NaN,administrative,34991901864,1693552721,1,"{""description"":""Das Gebiet des Ortsbezirks Hor...",relation,"POLYGON ((8.71890 49.57077, 8.71956 49.57011, ...",Hornbach,None,None,9,None,0.0


In [23]:
admin_boundaries = osm_data_obj.get_boundaries()
counties = admin_boundaries.loc[admin_boundaries['admin_level'] == '6']
counties

,visible,boundary,id,timestamp,version,geometry,tags,osm_type,name,ref,website,admin_level,border_type,changeset
4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.29988 49.47387, 8.29983 49.47397, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
5,NaN,administrative,8586762,1632578425,21,"POLYGON ((8.40363 49.30526, 8.40359 49.30541, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Speyer,None,None,6,None,0.0
6,NaN,administrative,8649216,1699197735,57,"MULTIPOLYGON (((8.15181 49.39231, 8.15294 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Landkreis Bad Dürkheim,None,None,6,county,0.0
7,NaN,administrative,8711789,1652953400,28,"POLYGON ((8.31071 49.50171, 8.31073 49.50210, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Frankenthal (Pfalz),None,None,6,None,0.0
8,NaN,administrative,8774366,1695228268,140,"MULTIPOLYGON (((8.60213 49.40930, 8.47028 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Rhein-Neckar-Kreis,None,None,6,None,0.0
9,NaN,administrative,8836976,1695967346,52,"POLYGON ((8.28099 49.58687, 8.28093 49.58408, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Rhein-Pfalz-Kreis,None,None,6,None,0.0
12,NaN,administrative,9024928,1662078812,55,"POLYGON ((8.41748 49.54873, 8.41752 49.54868, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Mannheim,None,None,6,None,0.0
13,NaN,administrative,9087652,1689220939,34,"MULTIPOLYGON (((8.14263 49.39446, 8.14514 49.3...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Neustadt an der Weinstraße,None,None,6,None,0.0
14,NaN,administrative,9150382,1693474719,120,"MULTIPOLYGON (((8.48176 49.57494, 8.49260 49.5...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Kreis Bergstraße,None,None,6,None,0.0
27,NaN,administrative,53278047,1695228268,32,"POLYGON ((8.57343 49.41576, 8.57329 49.41652, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0


In [44]:
school_filter = {"amenity": ["school"]}
schools = osm_data_obj.get_data_by_custom_criteria(custom_filter=school_filter)
schools_points = schools.copy()
schools_points['geometry']= schools.centroid
schools_points

/tmp/ipykernel_2788/725071667.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  schools_points['geometry']= schools.centroid


,lat,changeset,version,tags,timestamp,lon,visible,id,addr:city,addr:country,...,geometry,osm_type,addr:housename,ref,url,building:levels,internet_access,landuse,school,start_date
0,49.402039,0.0,3,None,1567840265,8.239435,False,272517372,Meckenheim,DE,...,POINT (8.23944 49.40204),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,49.565739,0.0,5,"{""wheelchair"":""no""}",1560223440,8.249144,False,295078565,None,None,...,POINT (8.24914 49.56574),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,49.508759,0.0,1,None,1228123254,8.508860,False,316019351,Mannheim,None,...,POINT (8.50886 49.50876),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,49.471767,0.0,5,"{""wheelchair"":""yes"",""wikidata"":""Q84995304""}",1595176296,8.655658,False,330955222,None,None,...,POINT (8.65566 49.47177),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,49.471947,0.0,6,"{""wheelchair"":""yes""}",1488664212,8.656594,False,333298502,None,None,...,POINT (8.65659 49.47195),node,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,NaN,0.0,1,"{""type"":""multipolygon"",""wikidata"":""Q2417212""}",1615918892,NaN,NaN,19869100895,None,None,...,POINT (8.42692 49.47811),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
467,NaN,0.0,1,"{""type"":""multipolygon""}",1615918892,NaN,NaN,19881555370,None,None,...,POINT (8.42586 49.47761),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
468,NaN,0.0,1,"{""short_name"":""KFG"",""type"":""multipolygon"",""wik...",1639739184,NaN,NaN,24682386538,None,None,...,POINT (8.69329 49.41086),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN
469,NaN,0.0,2,"{""operator:type"":""private"",""religion"":""christi...",1689361922,NaN,NaN,25833693929,Mannheim,None,...,POINT (8.46026 49.48605),relation,NaN,NaN,None,None,NaN,NaN,NaN,NaN


In [25]:
import h3pandas
hexgrid = counties.h3.polyfill_resample(10)
hexgrid

,index,visible,boundary,id,timestamp,version,geometry,tags,osm_type,name,ref,website,admin_level,border_type,changeset
h3_polyfill,,,,,,,,,,,,,,,
8a1fae610607fff,4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.36291 49.49548, 8.36266 49.49485, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
8a1fae61c0d7fff,4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.42683 49.50661, 8.42657 49.50598, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
8a1fae61e917fff,4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.40400 49.49909, 8.40375 49.49845, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
8a1fae6ab937fff,4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.39438 49.44539, 8.39413 49.44475, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
8a1fae6aab67fff,4,NaN,administrative,8524410,1695967346,38,"POLYGON ((8.37952 49.44199, 8.37927 49.44136, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Ludwigshafen am Rhein,None,None,6,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1faa99ed4ffff,27,NaN,administrative,53278047,1695228268,32,"POLYGON ((8.76571 49.40632, 8.76545 49.40568, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa995127fff,27,NaN,administrative,53278047,1695228268,32,"POLYGON ((8.73321 49.43779, 8.73295 49.43715, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faad65b07fff,27,NaN,administrative,53278047,1695228268,32,"POLYGON ((8.68125 49.40059, 8.68099 49.39995, ...","{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0


In [40]:
hexgrid_heidelberg = hexgrid.loc[hexgrid["name"] == 'Heidelberg']
hexgrid_heidelberg_centroids = hexgrid_heidelberg.copy()
hexgrid_heidelberg_centroids['geometry'] = hexgrid_heidelberg.centroid
hexgrid_heidelberg_centroids['id'] = range(53278047, 53278047 + len(hexgrid_heidelberg_centroids))
hexgrid_heidelberg_centroids

/tmp/ipykernel_2788/676297992.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hexgrid_heidelberg_centroids['geometry'] = hexgrid_heidelberg.centroid


,index,visible,boundary,id,timestamp,version,geometry,tags,osm_type,name,ref,website,admin_level,border_type,changeset
h3_polyfill,,,,,,,,,,,,,,,
8a1faad60057fff,27,NaN,administrative,53278047,1695228268,32,POINT (8.66531 49.37208),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa991b97fff,27,NaN,administrative,53278048,1695228268,32,POINT (8.75876 49.42270),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa9915a7fff,27,NaN,administrative,53278049,1695228268,32,POINT (8.74203 49.41257),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa996af7fff,27,NaN,administrative,53278050,1695228268,32,POINT (8.69974 49.41519),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa9b245ffff,27,NaN,administrative,53278051,1695228268,32,POINT (8.68175 49.44007),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8a1faa99ed4ffff,27,NaN,administrative,53285857,1695228268,32,POINT (8.76645 49.40591),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faa995127fff,27,NaN,administrative,53285858,1695228268,32,POINT (8.73395 49.43738),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0
8a1faad65b07fff,27,NaN,administrative,53285859,1695228268,32,POINT (8.68198 49.40018),"{""TMC:cid_58:tabcd_1:Class"":""Area"",""TMC:cid_58...",relation,Heidelberg,None,None,6,None,0.0


In [62]:
travel_time_matrix_computer = r5py.TravelTimeMatrixComputer(
    transport_network,
    origins= hexgrid_heidelberg_centroids,
    destinations= schools_points,
    departure=datetime.datetime(2022, 3, 27, 6, 30),
    transport_modes=[
        r5py.TransportMode.CAR],
)

school_run = travel_time_matrix_computer.compute_travel_times()
school_run

,from_id,to_id,travel_time
0,53278047,272517372,NaN
1,53278047,295078565,NaN
2,53278047,316019351,NaN
3,53278047,330955222,NaN
4,53278047,333298502,NaN
...,...,...,...
3680860,53285861,19869100895,NaN
3680861,53285861,19881555370,NaN
3680862,53285861,24682386538,NaN
3680863,53285861,25833693929,NaN


In [56]:
transit_layer = transport_network.transit_layer
transit_layer.start_date, transit_layer.end_date

(datetime.datetime(2022, 2, 22, 0, 0),
 datetime.datetime(2022, 4, 7, 23, 59, 59))

In [60]:
non_na_traveltimes = school_run[~school_run['travel_time'].isna()]
non_na_traveltimes

,from_id,to_id,travel_time
